In [12]:
from collections import deque
import numpy as np

In [13]:
from collections import deque
import numpy as np
class Car():
    def __init__(self):
        self.detected = False
        self.n_cars = deque(maxlen=10)
        self.bboxes = deque(maxlen=10)
        self.cars = [deque(maxlen=10) for i in np.arange(10)]
    
    def previous_centers(self):
        if len(self.bboxes[-1])>0:
            centers = find_centers(self.bboxes[-1])
        self.previous_centers = centers
        return centers


#     def calc_centers(bboxes):
#         centers=[]
#         for b in bboxes:
#             cx, cy = int((b[0]+b[2])/2), int((b[1]+b[3])/2)
#             centers.append((cx,cy))
#         return centers

    def calc_centers(bboxes):
        centers=[]
        for b in bboxes:
            cx, cy = int((b[0][0]+b[1][0])/2), int((b[0][1]+b[1][1])/2)
            centers.append((cx,cy))
        return centers

    def first_order_smooth(self, new_bb):

        if len(new_bb) == 0:
            self.detected = False
            self.n_cars = 0
            
        else: 
            new_centers = calc_centers(new_bb)
            cars = self.cars
            old_centers = [(i, calc_centers(d[-1])) for i,d in enumerate(cars) if len(d)>0]
            distances = []
            n_old = len(old_centers)
            n_new = len(new_centers)
            threshold = 20
            old_cars = self.cars
            
            if n_new <= n_old: 
                cursor = 0
                for id_nc, nc in enumerate(new_centers):
                    temps = []

                    if len(old_centers) >0: 
                        for id_oc, oc in enumerate(old_centers):
                            temp = np.sqrt(abs(oc[0]-nc[0])**2+abs(oc[1]-nc[1])**2)
                            temps.append(temp)


                        if min(temps) <= threshold: 

                            locate = temps.index(min(temps))

                            car_oc = old_centers[locate]

                            # first order smoothing
                            new_bb_refined = ( np.array(new_bb[id_nc])*0.6+
                                               np.mean(np.array(cars[locate+cursor]), axis=0)*0.4)
                            new_bb[id_nc] = tuple(map(tuple,new_bb_refined.astype(np.uint16)))
                            old_centers.pop(old_centers.index(old_centers[locate]))
                            cars[locate+cursor].append(new_bb[id_nc])
                            cursor += 1

                        else: 
                            cars.append([new_bb[id_nc]])

                    else: 
                        cars.append([new_bb[id_nc]])

                cars = [v for i,v in enumerate(cars) if v[-1] in new_bb]
                                                 
            else: 
                
                    while(len(old_centers)>0):
                        temps = []
                        for id_oc, oc in enumerate(old_centers):


                            if len(new_centers) >0: 
                                for id_nc, nc in enumerate(new_centers):
                                    temp = np.sqrt(abs(oc[0]-nc[0])**2+abs(oc[1]-nc[1])**2)
                                    temps.append(list(zip([temp], [oc],[[id_oc, id_nc]])))



                        if min([x[0][0] for x in temps]) <= threshold: 

                            locate = [y[0] for y in temps if y[0][0]==min([x[0][0] for x in temps])]

                            pop_old_loc = old_centers.index(old_centers[locate[0][2][0]])


                            new_bb_loc = new_centers_copy.index(new_centers[id_nc])
                            cars_loc = old_centers_copy.index(old_centers[id_oc])

                            new_bb_refined = np.array(new_bb[new_bb_loc])*0.6+np.mean(np.array(cars[cars_loc]), axis=0)*0.4
                            new_bb[id_nc] = tuple(map(tuple,new_bb_refined.astype(np.uint16)))


                            cars[cars_loc].append(new_bb[id_nc])

                            old_centers.pop(pop_old_loc)
                            new_centers.pop(locate[0][2][1])
                        else: 
                            break

                    for i in new_bb:
                        if i not in [c[-1] for c in cars]:
                            cars.append([i])

                    cars = [v for i,v in enumerate(cars) if v[-1] in new_bb]
    
            self.cars = cars
            self.bboxes.append([v[-1] for v in self.cars])
            self.detected = True
            self.n_cars = len(cars)
        
        return new_bb

In [14]:
def calc_centers(bboxes):
    centers=[]
    for b in bboxes:
        cx, cy = int((b[0][0]+b[1][0])/2), int((b[0][1]+b[1][1])/2)
        centers.append((cx,cy))
    return centers
new_bb = [((9,359),(448,530)), ((799,402),(1014,520))]
new_centers = calc_centers(new_bb)
print(new_centers)

[(228, 444), (906, 461)]


In [15]:
cars = [[((100,100),(200,200)),((109,100),(200,200))],
        [((790,402),(1014,520)),((795,402),(1014,520)),((798,402),(1014,520))]]
old_centers_history = [(i, calc_centers(d)) for i,d in enumerate(cars) if len(d)>0]
old_centers = [x[1][-1] for x in old_centers_history]
print(old_centers_history)
print(old_centers)

[(0, [(150, 150), (154, 150)]), (1, [(902, 461), (904, 461), (906, 461)])]
[(154, 150), (906, 461)]


In [16]:
distances = []
n_old = len(old_centers)
n_new = len(new_centers)
threshold = 20
print(n_old)
print(n_new)

2
2


In [17]:
print(cars)
cars_copy = cars
print(new_bb)

# done 
if n_new <= n_old: 
    cursor = 0
    for id_nc, nc in enumerate(new_centers):
        temps = []
        
        if len(old_centers) >0: 
            for id_oc, oc in enumerate(old_centers):
                temp = np.sqrt(abs(oc[0]-nc[0])**2+abs(oc[1]-nc[1])**2)
                temps.append(temp)
            
            if min(temps) <= threshold: 
                
                locate = temps.index(min(temps))
                
                car_oc = old_centers[locate]
                
                # first order smoothing
                new_bb_refined = np.array(new_bb[id_nc])*0.6+np.mean(np.array(cars[locate+cursor]), axis=0)*0.4
                new_bb[id_nc] = tuple(map(tuple,new_bb_refined.astype(np.uint16)))
                old_centers.pop(old_centers.index(car_oc))
                cars[locate+cursor].append(new_bb[id_nc])
                cursor += 1
                
            else: 
                cars.append([new_bb[id_nc]])
                
        else: 
            cars.append([new_bb[id_nc]])

    cars = [v for i,v in enumerate(cars) if v[-1] in new_bb]
#     self.cars = cars
#     self.bboxes.append([v[-1] for v in self.cars])

    

print(cars)

[[((100, 100), (200, 200)), ((109, 100), (200, 200))], [((790, 402), (1014, 520)), ((795, 402), (1014, 520)), ((798, 402), (1014, 520))]]
[((9, 359), (448, 530)), ((799, 402), (1014, 520))]
[[((790, 402), (1014, 520)), ((795, 402), (1014, 520)), ((798, 402), (1014, 520)), ((797, 402), (1014, 520))], [((9, 359), (448, 530))]]


In [18]:
new_bb = [((9,359),(448,530)), ((99,58),(55,99)), ((799,402),(1014,520))]
new_centers = calc_centers(new_bb)
new_centers_copy = new_centers[:]
print(new_bb)
print(new_centers)

cars = [[((100,100),(200,200)),((109,100),(200,200))],
        [((790,402),(1014,520)),((795,402),(1014,520)),((798,402),(1014,520))]]
#cars = [deque(maxlen=10) for i in np.arange(10)]
old_centers_history = [(i, calc_centers(d)) for i,d in enumerate(cars) if len(d)>0]
old_centers = [x[1][-1] for x in old_centers_history]
old_centers_copy = old_centers[:]
print(old_centers_history)
print(old_centers)

n_old = len(old_centers)
n_new = len(new_centers)
threshold = 20
print(n_old)
print(n_new)

[((9, 359), (448, 530)), ((99, 58), (55, 99)), ((799, 402), (1014, 520))]
[(228, 444), (77, 78), (906, 461)]
[(0, [(150, 150), (154, 150)]), (1, [(902, 461), (904, 461), (906, 461)])]
[(154, 150), (906, 461)]
2
3


In [19]:
if n_new > n_old: 

    while(len(old_centers)>0):
        temps = []
        for id_oc, oc in enumerate(old_centers):


            if len(new_centers) >0: 
                for id_nc, nc in enumerate(new_centers):
                    temp = np.sqrt(abs(oc[0]-nc[0])**2+abs(oc[1]-nc[1])**2)
                    temps.append(list(zip([temp], [oc],[[id_oc, id_nc]])))

    

        if min([x[0][0] for x in temps]) <= threshold: 

            locate = [y[0] for y in temps if y[0][0]==min([x[0][0] for x in temps])]

            pop_old_loc = old_centers.index(old_centers[locate[0][2][0]])


            new_bb_loc = new_centers_copy.index(new_centers[id_nc])
            cars_loc = old_centers_copy.index(old_centers[id_oc])

            new_bb_refined = np.array(new_bb[new_bb_loc])*0.6+np.mean(np.array(cars[cars_loc]), axis=0)*0.4
            new_bb[id_nc] = tuple(map(tuple,new_bb_refined.astype(np.uint16)))


            cars[cars_loc].append(new_bb[id_nc])

            old_centers.pop(pop_old_loc)
            new_centers.pop(locate[0][2][1])
        else: 
            break
        
    for i in new_bb:
        if i not in [c[-1] for c in cars]:
            cars.append([i])
    
    cars = [v for i,v in enumerate(cars) if v[-1] in new_bb]
    
    print(cars)

[[((790, 402), (1014, 520)), ((795, 402), (1014, 520)), ((798, 402), (1014, 520)), ((797, 402), (1014, 520))], [((9, 359), (448, 530))], [((99, 58), (55, 99))]]


In [20]:
len(cars[0])

4

In [21]:
[c[0] for c in cars]

[((790, 402), (1014, 520)), ((9, 359), (448, 530)), ((99, 58), (55, 99))]

In [22]:
old_centers

[(154, 150)]